In [2]:
import csv
import json
import random
import requests
import argparse
import pandas as pd
from datetime import datetime, timedelta, date
from dateutil.parser import parse
from icalendar import Calendar, Event, vCalAddress, vText
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import smtplib
import pytz

with open('global_config.json', 'r') as f:
    config = json.load(f)

SENDER_EMAIL = "prakhararora886@gmail.com"
SENDER_PASSWORD = "fqbr hvgx unjz lyzb"
MEETING_BASE_URL = "https://meet.example.com/interview"

def preprocess_candidate_data(excel_path: str) -> bool:
    try:
        df = pd.read_excel(excel_path, dtype={'Candidate_ID': str, 'Time_Zone': str})
        print(f"Original candidate rows: {len(df)}")
        
        # Clean and validate data
        df_clean = df.dropna(subset=['Candidate_ID', 'Raw_Input_Text', 'Time_Zone', 'Email'])
        valid_tz = set(pytz.all_timezones)
        df_clean = df_clean[df_clean['Time_Zone'].isin(valid_tz)]
        
        df_clean.to_csv("CandidateBasic.csv", index=False)
        print(f"Processed {len(df_clean)} candidates")
        return True
    except Exception as e:
        print(f"Candidate preprocessing failed: {str(e)}")
        return False

def preprocess_recruiter_data(excel_path: str) -> bool:
    try:
        df = pd.read_excel(excel_path, dtype={'Recruiter_ID': str, 'Time_Zone': str})
        print(f"Original recruiter rows: {len(df)}")
        
        df_clean = df.dropna(subset=['Recruiter_ID', 'Weekly_Availability', 'Time_Zone', 'Email'])
        df_clean['Weekly_Availability'] = df_clean['Weekly_Availability'].apply(
            lambda x: json.dumps({day: [s.strip(' "\'') for s in x.split(",")] 
                                for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]})
        )
        
        valid_tz = set(pytz.all_timezones)
        df_clean = df_clean[df_clean['Time_Zone'].isin(valid_tz)]
        
        df_clean.to_csv("RecruiterAvailability.csv", index=False)
        print(f"Processed {len(df_clean)} recruiters")
        return True
    except Exception as e:
        print(f"Recruiter preprocessing failed: {str(e)}")
        return False

def generate_candidate_csv():
    candidates = []
    timezones = ['US/Eastern', 'US/Pacific', 'US/Central', 'UTC']
    
    for cand_id in range(1, config["num_candidates"] + 1):
        candidates.append({
            "Candidate_ID": cand_id,
            "Raw_Input_Text": random.choice(config["candidate_templates"]).format(
                slot=random.choice(["10:00-10:30", "11:00-11:30", "13:00-13:30", "14:00-14:30", "15:00-15:30"])
            ),
            "Time_Zone": 'US/Eastern' if random.random() < 0.6 else random.choice(timezones[1:]),
            "Email": "prakhararora60@gmail.com"
        })
    
    with open("CandidateBasic.csv", "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["Candidate_ID", "Raw_Input_Text", "Time_Zone", "Email"])
        writer.writeheader()
        writer.writerows(candidates)

def process_temporal_with_duckling(text: str, reference_time: datetime) -> list:
    try:
        response = requests.post(
            "http://localhost:8000/parse",
            params={"locale": "en_US", "dims": '["time"]', "reftime": reference_time.isoformat()},
            data={"text": text}
        )
        return response.json() if response.status_code == 200 else []
    except requests.RequestException:
        return []

def update_candidate_csv_with_duckling():
    updated = []
    ref_date = next((date.today() + timedelta(days=i) for i in range(1,8) 
               if (date.today() + timedelta(days=i)).weekday() == 0))
    
    with open("CandidateBasic.csv", "r") as f:
        for row in csv.DictReader(f):
            tz = pytz.timezone(row["Time_Zone"])
            parsed = process_temporal_with_duckling(
                row["Raw_Input_Text"],
                tz.localize(datetime.combine(ref_date, datetime.min.time())))
            
            converted_day = "None"
            converted_slot = "None"
            duckling_used = False
            
            if parsed:
                duckling_used = True
                for p in parsed:
                    if p['dim'] == 'time' and 'from' in p['value']:
                        try:
                            start = parse(p['value']['from']['value'])
                            end = parse(p['value']['to']['value']) if 'to' in p['value'] else \
                                  start + timedelta(minutes=config["meeting_duration_minutes"])
                            
                            est = pytz.timezone('US/Eastern')
                            start_est = start.astimezone(est)
                            end_est = end.astimezone(est)
                            
                            converted_day = start_est.strftime("%A")
                            converted_slot = f"{start_est.strftime('%H:%M')}-{end_est.strftime('%H:%M')}"
                            break
                        except KeyError as e:
                            continue
            
            updated.append({
                **row,
                "Converted_Day": converted_day,
                "Converted_Slot": converted_slot,
                "Duckling_Used": duckling_used
            })
    
    with open("CandidateUpdated.csv", "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=updated[0].keys())
        writer.writeheader()
        writer.writerows(updated)

def generate_recruiter_csv():
    work_start = datetime.strptime(config["work_hours"]["start"], "%H:%M").time()
    work_end = datetime.strptime(config["work_hours"]["end"], "%H:%M").time()
    
    # Generate base time slots
    base_slots = []
    current = datetime.combine(date.today(), work_start)
    end_day = datetime.combine(date.today(), work_end)
    
    while current < end_day:
        slot_end = current + timedelta(minutes=config["meeting_duration_minutes"])
        if not any(current.time() < br_end and slot_end.time() > br_start 
                 for br_start, br_end in [
                    (datetime.strptime(t.split('-')[0], "%H:%M").time(),
                     datetime.strptime(t.split('-')[1], "%H:%M").time())
                    for t in config["breaks"].values()]):
            base_slots.append(f"{current.time().strftime('%H:%M')}-{slot_end.time().strftime('%H:%M')}")
        current = slot_end
    
    recruiters = []
    for rec_id in range(1, config["num_recruiters"] + 1):
        weekly_availability = {
            day: [slot for slot in base_slots if random.random() > 0.2]
            for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
        }
        recruiters.append({
            "Recruiter_ID": f"R{rec_id}",
            "Weekly_Availability": json.dumps(weekly_availability),
            "Time_Zone": config["default_recruiter_timezone"],
            "Email": "prakhararora886@gmail.com"
        })
    
    with open("RecruiterAvailability.csv", "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["Recruiter_ID", "Weekly_Availability", "Time_Zone", "Email"])
        writer.writeheader()
        writer.writerows(recruiters)

def schedule_events_from_csv():
    
    candidates = []
    with open("CandidateUpdated.csv", "r") as f:
        candidates = list(csv.DictReader(f))
    
    recruiters = []
    with open("RecruiterAvailability.csv", "r") as f:
        recruiters = list(csv.DictReader(f))
    
    availability_pool = []
    day_order = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Friday": 4}
    
    for rec in recruiters:
        availability = json.loads(rec["Weekly_Availability"])
        for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]:
            sorted_slots = sorted(availability[day], key=lambda x: x.split('-')[0])
            for slot in sorted_slots:
                availability_pool.append({
                    "recruiter_id": rec["Recruiter_ID"],
                    "email": rec["Email"],
                    "day": day,
                    "slot": slot,
                    "start_time": datetime.strptime(slot.split('-')[0], "%H:%M").time(),
                    "scheduled": False
                })
    
    availability_pool.sort(key=lambda x: (day_order[x["day"]], x["start_time"]))
    
    events = []
    event_id = 1
    
    for cand in candidates:
        scheduled = False
        
        if cand["Converted_Day"] != "None" and cand["Converted_Slot"] != "None":
            preferred_day = cand["Converted_Day"]
            preferred_slot = cand["Converted_Slot"]
            
            for slot in availability_pool:
                if (not slot["scheduled"] and 
                    slot["day"] == preferred_day and 
                    slot["slot"] == preferred_slot):
                    
                    events.append(create_event(event_id, cand, slot))
                    slot["scheduled"] = True
                    event_id += 1
                    scheduled = True
                    break
        
        if not scheduled:
            for slot in availability_pool:
                if not slot["scheduled"]:
                    events.append(create_event(event_id, cand, slot))
                    slot["scheduled"] = True
                    event_id += 1
                    scheduled = True
                    break
        
        if not scheduled:
            events.append({
                "Event_ID": event_id,
                "Candidate_ID": cand["Candidate_ID"],
                "Candidate_Email": cand["Email"],
                "Recruiter_ID": "None",
                "Recruiter_Email": "",
                "Day": "None",
                "Scheduled_Time_Slot": "None",
                "Outcome": "Failed"
            })
            event_id += 1
    
    with open("SchedulingEvents.csv", "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=events[0].keys())
        writer.writeheader()
        writer.writerows(events)
    
    return events

def create_event(event_id, candidate, slot):
    return {
        "Event_ID": event_id,
        "Candidate_ID": candidate["Candidate_ID"],
        "Candidate_Email": candidate["Email"],
        "Recruiter_ID": slot["recruiter_id"],
        "Recruiter_Email": slot["email"],
        "Day": slot["day"],
        "Scheduled_Time_Slot": slot["slot"],
        "Outcome": "Confirmed"
    }
SENDER_EMAIL = "prakhararora886@gmail.com"
SENDER_PASSWORD = "fqbr hvgx unjz lyzb"
MEETING_BASE_URL = "https://meet.example.com/interview"

def send_calendar_invites():
    # Read scheduled events
    with open('SchedulingEvents.csv', 'r') as f:
        events = list(csv.DictReader(f))
    
    ref_date = date.today()
    while ref_date.weekday() != 0:
        ref_date += timedelta(days=1)

    for event in events:
        if event['Outcome'] != 'Confirmed':
            continue

        # datetime objects
        day_name = event['Day']
        days_offset = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"].index(day_name)
        event_date = ref_date + timedelta(days=days_offset)
        
        start_time_str, end_time_str = event['Scheduled_Time_Slot'].split('-')
        start_time = datetime.strptime(start_time_str, "%H:%M").time()
        end_time = datetime.strptime(end_time_str, "%H:%M").time()
        
        est = pytz.timezone('US/Eastern')
        start_datetime = est.localize(datetime.combine(event_date, start_time))
        end_datetime = est.localize(datetime.combine(event_date, end_time))

        cal = Calendar()
        cal.add('prodid', '-//Interview Scheduler//')
        cal.add('version', '2.0')

        ical_event = Event()
        ical_event.add('summary', 'Interview Session')
        ical_event.add('dtstart', start_datetime)
        ical_event.add('dtend', end_datetime)
        ical_event.add('location', MEETING_BASE_URL + f"/{event['Event_ID']}")
        ical_event.add('description', f"Interview between Candidate {event['Candidate_ID']} and Recruiter {event['Recruiter_ID']}")

        organizer = vCalAddress(f'mailto:{SENDER_EMAIL}')
        organizer.params['cn'] = vText('Scheduling System')
        ical_event['organizer'] = organizer

        for email in [event['Candidate_Email'], event['Recruiter_Email']]:
            attendee = vCalAddress(f'mailto:{email}')
            attendee.params['ROLE'] = vText('REQ-PARTICIPANT')
            ical_event.add('attendee', attendee, encode=0)

        cal.add_component(ical_event)

        for recipient in [event['Candidate_Email'], event['Recruiter_Email']]:
            try:
                msg = MIMEMultipart()
                msg['From'] = SENDER_EMAIL
                msg['To'] = recipient  
                msg['Subject'] = f"Interview Scheduled - {day_name} {event['Scheduled_Time_Slot']}"

                role = "Candidate" if recipient == event['Candidate_Email'] else "Recruiter"
                body = f"""<html>
                    <body>
                        <h2>Interview Confirmed ({role})</h2>
                        <p>Date: {event_date.strftime('%Y-%m-%d')} ({day_name})</p>
                        <p>Time: {event['Scheduled_Time_Slot']}</p>
                        <p>Meeting Link: <a href="{MEETING_BASE_URL}/{event['Event_ID']}">Join Meeting</a></p>
                        <p>Calendar invite attached</p>
                    </body>
                </html>"""
                
                msg.attach(MIMEText(body, 'html'))
                
                part = MIMEBase('text', "calendar", name="event.ics")
                part.set_payload(cal.to_ical())
                encoders.encode_base64(part)
                part.add_header('Content-Disposition', 'attachment; filename="interview.ics"')
                msg.attach(part)

                with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
                    server.login(SENDER_EMAIL, SENDER_PASSWORD)
                    server.sendmail(SENDER_EMAIL, recipient, msg.as_string())
                print(f"Sent invite to {recipient}")
                
            except Exception as e:
                print(f"Failed to send to {recipient}: {str(e)}")

def main(candidate_file: str = None, recruiter_file: str = None):
    if candidate_file:
        if not preprocess_candidate_data(candidate_file):
            return
        update_candidate_csv_with_duckling()
    else:
        generate_candidate_csv()
        update_candidate_csv_with_duckling()
    
    if recruiter_file:
        if not preprocess_recruiter_data(recruiter_file):
            return
    else:
        generate_recruiter_csv()
    
    events = schedule_events_from_csv()
    send_calendar_invites()
    
    print("\nScheduling Complete:")
    print(f"Total Events: {len(events)}")
    print(f"Successful: {sum(1 for e in events if e['Outcome'] == 'Confirmed')}")
    print(f"Failed: {sum(1 for e in events if e['Outcome'] == 'Failed')}")

if __name__ == "__main__":
    main(candidate_file=None,recruiter_file=None)
    #main(candidate_file="path/to/candidates.xlsx", recruiter_file="path/to/recruiters.xlsx")

Sent invite to prakhararora60@gmail.com
Sent invite to prakhararora886@gmail.com
Sent invite to prakhararora60@gmail.com
Sent invite to prakhararora886@gmail.com
Sent invite to prakhararora60@gmail.com
Sent invite to prakhararora886@gmail.com
Sent invite to prakhararora60@gmail.com
Sent invite to prakhararora886@gmail.com
Sent invite to prakhararora60@gmail.com
Sent invite to prakhararora886@gmail.com
Sent invite to prakhararora60@gmail.com
Sent invite to prakhararora886@gmail.com
Sent invite to prakhararora60@gmail.com
Sent invite to prakhararora886@gmail.com
Sent invite to prakhararora60@gmail.com
Sent invite to prakhararora886@gmail.com
Sent invite to prakhararora60@gmail.com
Sent invite to prakhararora886@gmail.com
Sent invite to prakhararora60@gmail.com
Sent invite to prakhararora886@gmail.com

Scheduling Complete:
Total Events: 10
Successful: 10
Failed: 0
